# Getting data

In [2080]:
#   Importing libraries
from bs4 import BeautifulSoup
import re   #regular expressions
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datetime as dt

import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV

%matplotlib inline

In [ ]:
from Project_2_functions.py import get_name, get_expansion_rarity_clean, \
get_price_clean, get_mana_costs, get_converted_cost, get_card_color, \
get_rules_text, all_scrape_modern, get_expansion_names_dates, \
remove_bracket_rel_dates, scrape_core_sets, remove_brackets_core_set_list, \
scrape_reprint_comp_sets, remove_brackets_supp_sets, get_days


## Connecting to Card Kingdom modern format page

In [374]:
# Sending request to Card Kingdom

modern_url = 'https://www.cardkingdom.com/catalog/view?filter%5Bsort%5D=most_popular&filter%5Bsearch%5D=mtg_advanced&filter%5Btab%5D=&filter%5Bname%5D=&filter%5Bcategory_id%5D=2864&filter%5Bmulti%5D%5B0%5D=1&filter%5Btype_mode%5D=any&filter%5Btype_key%5D=&filter%5Bpow1%5D=&filter%5Bpow2%5D=&filter%5Btuf1%5D=&filter%5Btuf2%5D=&filter%5Bconcast1%5D=&filter%5Bconcast2%5D=&filter%5Bprice_op%5D=&filter%5Bprice%5D=&filter%5Bkey_text1%5D=&filter%5Bmanaprod_select%5D=any' 

response = requests.get(standard_url)


In [27]:
response.status_code  #200 = success!

200

In [28]:
response.text[:1000]  #First 1000 characters of the HTML

'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta http-equiv="Cache-control" content="max-age=29030400,public">\n<meta name="description" content="Magic The Gathering, magic cards, singles, decks, card lists, deck ideas, wizard of the coast, all of the cards you need at great prices are available at Cardkingdom.">\n<meta name="keywords" content="Magic The Gathering, magic cards, magic the gathering and purchase, magic singles, magic the gathering pricelist, magic the gathering card lists, magic the gathering deck ideas, magic the gathering cards, magic the gathering cards sell, magic the gathering card prices, wizard of the coast, magic the gathering singles, magic the gathering decks, magic the gathering cardlist, cheap magic singles.">\n<meta NAME="robots" CONTENT="noindex, follow">\n<meta name="google-site-verification" content="mG_HnQSIpvWtvlgoloS5nWbuB7wNauY3RlU6dEmTLec" />\n<meta http-equiv="X-UA-Compatible" c

### Creating soup object

This is redone in all_scrape() function below

In [375]:
page = response.text
soup = BeautifulSoup(page)

In [ ]:
print(soup.prettify)

### Functions to scrape variables

### Using driver to navigate pages

In [314]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [335]:
driver = webdriver.Chrome(chromedriver)
first_page_url = 'https://www.cardkingdom.com/catalog/view?filter%5Bsort%5D=most_popular&filter%5Bsearch%5D=mtg_advanced&filter%5Btab%5D=&filter%5Bname%5D=&filter%5Bcategory_id%5D=2779&filter%5Bmulti%5D%5B0%5D=1&filter%5Btype_mode%5D=any&filter%5Btype_key%5D=&filter%5Bpow1%5D=&filter%5Bpow2%5D=&filter%5Btuf1%5D=&filter%5Btuf2%5D=&filter%5Bconcast1%5D=&filter%5Bconcast2%5D=&filter%5Bprice_op%5D=&filter%5Bprice%5D=&filter%5Bkey_text1%5D=&filter%5Bmanaprod_select%5D=any'
driver.get(first_page_url)

In [336]:
number_of_pages = int(driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[5]/div[2]/ul/li[13]/a").text)
number_of_pages

100

## Modern format

In [1247]:
modern_url = 'https://www.cardkingdom.com/catalog/view?filter%5Bsort%5D=most_popular&filter%5Bsearch%5D=mtg_advanced&filter%5Btab%5D=&filter%5Bname%5D=&filter%5Bcategory_id%5D=2864&filter%5Bmulti%5D%5B0%5D=1&filter%5Btype_mode%5D=any&filter%5Btype_key%5D=&filter%5Bpow1%5D=&filter%5Bpow2%5D=&filter%5Btuf1%5D=&filter%5Btuf2%5D=&filter%5Bconcast1%5D=&filter%5Bconcast2%5D=&filter%5Bprice_op%5D=&filter%5Bprice%5D=&filter%5Bkey_text1%5D=&filter%5Bmanaprod_select%5D=any' 

modern_response = requests.get(modern_url)

In [1248]:
modern_response.status_code  #200 = success!

200

### Creating modern soup

In [1249]:
modern_page = modern_response.text
modern_soup = BeautifulSoup(modern_page)

### Scraping multiple pages with chromedriver

In [1250]:
driver = webdriver.Chrome(chromedriver)
modern_first_page_url = 'https://www.cardkingdom.com/catalog/view?filter%5Bsort%5D=most_popular&filter%5Bsearch%5D=mtg_advanced&filter%5Btab%5D=&filter%5Bname%5D=&filter%5Bcategory_id%5D=2864&filter%5Bmulti%5D%5B0%5D=1&filter%5Btype_mode%5D=any&filter%5Btype_key%5D=&filter%5Bpow1%5D=&filter%5Bpow2%5D=&filter%5Btuf1%5D=&filter%5Btuf2%5D=&filter%5Bconcast1%5D=&filter%5Bconcast2%5D=&filter%5Bprice_op%5D=&filter%5Bprice%5D=&filter%5Bkey_text1%5D=&filter%5Bmanaprod_select%5D=any' 

driver.get(modern_first_page_url)

### Scraping all pages with function

In [1350]:
name, expansion, rarity, mana_cost, cmc, color_identity, rules_text, price = all_scrape_modern()


In [1352]:
len(expansion)

18087

### Creating dataframe

In [1398]:
data_tuples_modern = list(zip(name, expansion, rarity, mana_cost, cmc, color_identity, rules_text, price))

modern_df = pd.DataFrame(data_tuples_modern, columns = ['name','expansion', 'rarity', 'cost', 'cmc', 'color_identity', 'rules_text', 'price'])

# df.set_index('name', drop=True, inplace = True)

modern_df.head()

,name,expansion,rarity,cost,cmc,color_identity,rules_text,price
0,Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99
1,Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99
2,Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99
3,Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99
4,Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99


In [1399]:
modern_df.shape

(18087, 8)

### Creating pickle of modern data just in case

In [1355]:
filename = 'modern_data_baseline_PICKLE'
outfile = open(filename,'wb')
pickle.dump(modern_df,outfile)
outfile.close()

# Data cleaning modern dataset

In [2340]:
modern_df = pd.read_pickle('modern_data_baseline_PICKLE')

modern_df.head()

,name,expansion,rarity,cost,cmc,color_identity,rules_text,price
0,Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99
1,Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99
2,Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99
3,Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99
4,Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99


In [2341]:
# Getting rid of special rarities (showcase etc) and basic lands (L)

mask = (modern_df['rarity'] != 'L') & (modern_df['rarity'] != 'S')

modern_df = modern_df[mask]



In [2342]:
modern_df.shape

(16494, 8)

## Removing duplicates: Extended Art, Borderless, Showcase, Godzilla Series

In [2259]:
modern_df.head()

,name,expansion,rarity,cost,cmc,color_identity,rules_text,price
0,Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99
1,Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99
2,Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99
3,Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99
4,Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99


In [2343]:
# removing special versions of cards as above

ex_art_mask = ~modern_df['name'].str.contains("Extended Art")

modern_df = modern_df[ex_art_mask] # ~ means 'not'

borderless_mask = ~modern_df['name'].str.contains("Borderless")

modern_df = modern_df[borderless_mask] 

showcase_mask = ~modern_df['name'].str.contains("Showcase")

modern_df = modern_df[showcase_mask]

godzilla_mask = ~modern_df['name'].str.contains("Godzilla")

modern_df = modern_df[godzilla_mask]

### Setting index back to name

In [2344]:
modern_df.set_index('name', drop=True, inplace = True)


modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price
name,,,,,,,
Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99
Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99
Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99
Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99


In [2345]:
modern_df.shape

(16141, 7)

## fixing color identity

In [2346]:
# colorless cards were 'n' so making replacement

modern_df['color_identity'].replace('n', 'colorless', inplace=True)
modern_df['color_identity'].replace('e', 'colorless', inplace=True)

modern_df.shape

(16141, 7)

## Coding color ID to 5 cols

Color identity

In [2347]:
is_w = []
is_u = []
is_b = []
is_r = []
is_g = []

for card_color_identity in modern_df['color_identity']:
    if card_color_identity != 'colorless':
        if 'w' in card_color_identity:
            is_w.append(1)
        else: 
            is_w.append(0)

        if 'u' in card_color_identity:
            is_u.append(1)
        else: 
            is_u.append(0)

        if 'b' in card_color_identity:
            is_b.append(1)
        else: 
            is_b.append(0)

        if 'r' in card_color_identity:
            is_r.append(1)
        else: 
            is_r.append(0)

        if 'g' in card_color_identity:
            is_g.append(1)
        else: 
            is_g.append(0)
    else: 
        is_w.append(0)
        is_u.append(0)
        is_b.append(0)
        is_r.append(0)
        is_g.append(0)
        
# print('w:', is_w)
# print('u:', is_u)
# print('b:', is_b)

data_tuples = list(zip(is_w, is_u, is_b, is_r, is_g))
color_var_df = pd.DataFrame(data_tuples, columns = ['is_w', 'is_u', 'is_b', 'is_r', 'is_g'], index=modern_df.index)

color_var_df.head()

,is_w,is_u,is_b,is_r,is_g
name,,,,,
Gideon of the Trials,1,0,0,0,0
Force of Negation,0,1,0,0,0
Polluted Delta,0,0,0,0,0
Cavern of Souls,0,0,0,0,0
Wooded Foothills,0,0,0,0,0


In [2348]:
# Adding back to frame

modern_df = pd.concat([modern_df, color_var_df], axis=1, sort=False)

modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g
name,,,,,,,,,,,,
Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99,1,0,0,0,0
Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99,0,1,0,0,0
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99,0,0,0,0,0
Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99,0,0,0,0,0
Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99,0,0,0,0,0


In [2349]:
# Did it work? - yes!

modern_df.loc['Wrenn and Six']

expansion                                           Modern Horizons
rarity                                                            M
cost                                                             rg
cmc                                                               2
color_identity                                                   gr
rules_text        [+1]: Return up to one target land card from y...
price                                                         59.99
is_w                                                              0
is_u                                                              0
is_b                                                              0
is_r                                                              1
is_g                                                              1
Name: Wrenn and Six, dtype: object

## Adding rarity num column 0-4

In [2350]:
modern_df['rarity_num'] = modern_df.rarity.apply(lambda x: get_rarity_num(x))
modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num
name,,,,,,,,,,,,,
Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99,1,0,0,0,0,4
Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99,0,1,0,0,0,3
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99,0,0,0,0,0,3
Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99,0,0,0,0,0,3
Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99,0,0,0,0,0,3


## Adding rules text length column

In [2351]:
rules_text_len_mod = []
for i in modern_df['rules_text']:
    rules_text_len_mod.append(len(i))
    
modern_df['rules_text_len'] = rules_text_len_mod
modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len
name,,,,,,,,,,,,,,
Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99,1,0,0,0,0,4,401
Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99,0,1,0,0,0,3,234
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99,0,0,0,0,0,3,136
Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99,0,0,0,0,0,3,204
Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99,0,0,0,0,0,3,140


## Saving df with rarity and color dummies as pickle just in case

In [2270]:
filename = 'modern_data_rarity_color_rules_len_coded_10-8-20_PICKLE'
outfile = open(filename,'wb')
pickle.dump(modern_df,outfile)
outfile.close()


## Looking at how many times a card has been printed

In [2352]:
# making dict of all value counts
num_printings_dict = modern_df.index.value_counts().to_dict()

Note: this fails to see supplemental sets because those set lists are not included in analysis

In [2353]:
# adding dict to df
modern_df['num_printings']= modern_df.index.map(num_printings_dict)

modern_df.head(30)

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len,num_printings
name,,,,,,,,,,,,,,,
Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99,1,0,0,0,0,4,401,1
Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99,0,1,0,0,0,3,234,1
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99,0,0,0,0,0,3,136,1
Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99,0,0,0,0,0,3,204,1
Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99,0,0,0,0,0,3,140,1
Bloodstained Mire,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Bloodstained Mire: Sea...",29.99,0,0,0,0,0,3,151,1
Wrenn and Six,Modern Horizons,M,rg,2,gr,[+1]: Return up to one target land card from y...,59.99,0,0,0,1,1,4,216,1
Smothering Tithe,Ravnica Allegiance,R,3w,4,w,"Whenever an opponent draws a card, that player...",20.99,1,0,0,0,0,3,187,1
Windswept Heath,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Windswept Heath: Searc...",19.99,0,0,0,0,0,3,137,1


## Adding since release variable

### Connecting to wikipedia 

In [2273]:
# time_since_release_dict = {}

# time_since_release_dict['']

In [2274]:
# challenge: making "time_since_release" variable: 
# need to make new series that maps a year (or month) val to 'expansion' series

In [2400]:
# scraping wikipedia?

mtg_set_wiki_url = 'https://en.wikipedia.org/wiki/List_of_Magic:_The_Gathering_sets'

response = requests.get(mtg_set_wiki_url)

response.status_code  #200

200

In [2401]:
response.text[:1000]  #First 1000 characters of the HTML

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of Magic: The Gathering sets - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"02fb2cf1-069c-4d55-8506-d3f63de4c04c","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Magic:_The_Gathering_sets","wgTitle":"List of Magic: The Gathering sets","wgCurRevisionId":978729560,"wgRevisionId":978729560,"wgArticleId":507735,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with dead external links","Articles with dead external links from January 2020","Articles w

Making soup object

In [2402]:
page = response.text
soup = BeautifulSoup(page)

print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Magic: The Gathering sets - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"02fb2cf1-069c-4d55-8506-d3f63de4c04c","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Magic:_The_Gathering_sets","wgTitle":"List of Magic: The Gathering sets","wgCurRevisionId":978729560,"wgRevisionId":978729560,"wgArticleId":507735,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with dead external links","Articles with dead external links from J

### Scraping expansion names and release dates from wikipedia

Main, standard-legal expansions

In [2404]:
expansion_names, set_dates = get_expansion_names_dates(soup)

Arabian Nights

December 1993[31]



Antiquities

March 1994[32]



Legends

June 1994[33]



The Dark

August 1994[34]



Fallen Empires

November 1994[35]



Ice Age

June 1995[36]



Homelands[IX]

October 1995[37]



Alliances

June 10, 1996[10]



Mirage

October 7, 1996[10]



Visions

February 3, 1997[41]



Weatherlight

June 9, 1997[10]



Tempest

October 13, 1997[10]



Stronghold

March 2, 1998[10]



Exodus

June 15, 1998[10]



Urza's Saga

October 12, 1998[47]



Urza's Legacy

February 15, 1999[10]



Urza's Destiny

June 7, 1999[49]



Mercadian Masques

October 4, 1999[51]



Nemesis

February 14, 2000[53]



Prophecy

June 5, 2000[54]



Invasion

October 2, 2000[55]



Planeshift

February 5, 2001[56]



Apocalypse

June 4, 2001



Odyssey

October 1, 2001



Torment

February 4, 2002[58]



Judgment

May 27, 2002[59]



Onslaught

October 7, 2002[60]



Legions

February 3, 2003



Scourge

May 26, 2003[62]



Mirrodin

October 3, 2003[63]



Darksteel

February 6,

In [2405]:
print(len(expansion_names), len(set_dates))

85 85


In [2406]:
# Removing [] from release dates

set_dates_new = remove_bracket_rel_dates(set_dates)
len(set_dates_new)

85

Making dataframe of standard-legal sets

In [2408]:
# make sure dats and names are same length

len(set_dates_new), len(expansion_names)

(85, 85)

In [2409]:
sets_and_rel_dates = pd.DataFrame(list(zip(expansion_names, set_dates_new)), columns = ['expansion', 'rel_date'])

sets_and_rel_dates.head()

,expansion,rel_date
0,Arabian Nights,December 1993
1,Antiquities,March 1994
2,Legends,June 1994
3,The Dark,August 1994
4,Fallen Empires,November 1994


Repeating process for core sets

In [2410]:
# Scraping set and release date: core sets

core_expansion_names, core_set_dates = scrape_core_sets(soup)

Limited Edition Alpha
August 5, 1993[6]

Limited Edition Beta
October 1993[7]

Unlimited Edition
December 1993[6]

Revised Edition
April 1994[8]

Fourth Edition
April 1995[9]

Fifth Edition
March 24, 1997[10]

Classic Sixth Edition
April 28, 1999[10]

Seventh Edition
April 11, 2001[10]

Eighth Edition
July 28, 2003[11]

Ninth Edition
July 29, 2005[12]

Tenth Edition
July 13, 2007[13]

Magic 2010
July 17, 2009[15]

Magic 2011
July 16, 2010[16]

Magic 2012
July 15, 2011[17]

Magic 2013
July 13, 2012[18]

Magic 2014
July 19, 2013[19]

Magic 2015
July 18, 2014[20]

Magic Origins
July 17, 2015[22]

Core Set 2019
July 13, 2018[23]

Core Set 2020
July 12, 2019[24]

Core Set 2021
July 3, 2020[25]



In [2411]:
len(core_expansion_names)

21

In [2412]:
# Removing [] from core release dates

core_set_dates_new = remove_brackets_core_set_list(core_set_dates)

['August 5, 1993',
 'October 1993',
 'December 1993',
 'April 1994',
 'April 1995',
 'March 24, 1997',
 'April 28, 1999',
 'April 11, 2001',
 'July 28, 2003',
 'July 29, 2005',
 'July 13, 2007',
 'July 17, 2009',
 'July 16, 2010',
 'July 15, 2011',
 'July 13, 2012',
 'July 19, 2013',
 'July 18, 2014',
 'July 17, 2015',
 'July 13, 2018',
 'July 12, 2019',
 'July 3, 2020']

In [2413]:
# Making core set frame

core_sets_and_rel_dates = pd.DataFrame(list(zip(core_expansion_names, core_set_dates_new)), columns = ['expansion', 'rel_date'])
core_sets_and_rel_dates.head()

,expansion,rel_date
0,Limited Edition Alpha,"August 5, 1993"
1,Limited Edition Beta,October 1993
2,Unlimited Edition,December 1993
3,Revised Edition,April 1994
4,Fourth Edition,April 1995


Repeating for compilations/reprint sets

In [2414]:
# Scraping set and release date: supp sets

supp_expansion_names, supp_set_dates = scrape_reprint_comp_sets(soup)

Chronicles[XVII]

July 1995[153]



Rivals Quick Start Set

July 1996[154]



Multiverse Gift Box

November 1996[155]



Anthologies

November 1998[156]



Battle Royale Box Set

November 12, 1999[157]



Beatdown Box Set

December 2000[158]



Deckmasters: Garfield vs. Finkel

September 17, 2001[159]



Premium Foil Booster

January 8, 2010[160]



Duels of the Planeswalkers (decks)

June 4, 2010[161]



Modern Event Deck

May 30, 2014[162]



Renaissance (French/German)

August 1995[153]



Rinascimento (Italian)

August 1995[153]



Duel Decks: Elves vs. Goblins

November 16, 2007[163]



Duel Decks: Jace vs. Chandra

November 7, 2008[164]



Duel Decks: Divine vs. Demonic

April 10, 2009[165]



Duel Decks: Garruk vs. Liliana

October 30, 2009[166]



Duel Decks: Phyrexia vs. the Coalition

March 19, 2010[167]



Duel Decks: Elspeth vs. Tezzeret

September 3, 2010[168]



Duel Decks: Knights vs. Dragons

April 1, 2011[169]



Duel Decks: Ajani vs. Nicol Bolas

September 2, 2011[170

In [2415]:
# Removing [] from supp release dates

supp_set_dates_new = remove_brackets_supp_sets(supp_set_dates)

['July 1995',
 'July 1996',
 'November 1996',
 'November 1998',
 'November 12, 1999',
 'December 2000',
 'September 17, 2001',
 'January 8, 2010',
 'June 4, 2010',
 'May 30, 2014',
 'August 1995',
 'August 1995',
 'November 16, 2007',
 'November 7, 2008',
 'April 10, 2009',
 'October 30, 2009',
 'March 19, 2010',
 'September 3, 2010',
 'April 1, 2011',
 'September 2, 2011',
 'March 30, 2012',
 'September 7, 2012',
 'March 15, 2013',
 'September 6, 2013',
 'March 14, 2014',
 'September 5, 2014',
 'December 5, 2014',
 'February 27, 2015',
 'August 28, 2015',
 'February 26, 2016',
 'September 2, 2016',
 'March 31, 2017',
 'November 10, 2017',
 'April 6, 2018',
 'August 29, 2008',
 'August 28, 2009',
 'August 27, 2010',
 'August 26, 2011',
 'August 31, 2012',
 'August 23, 2013',
 'August 22, 2014',
 'August 21, 2015',
 'August 19, 2016',
 'November 24, 2017',
 'June 15, 2018',
 'June 28, 2019',
 'November 20, 2009',
 'November 19, 2010',
 'November 18, 2011',
 'June 7, 2013',
 'May 22, 201

In [2416]:
# Making supp set frame

supp_sets_and_rel_dates = pd.DataFrame(list(zip(supp_expansion_names, supp_set_dates_new)), columns = ['expansion', 'rel_date'])
supp_sets_and_rel_dates.head()

,expansion,rel_date
0,Chronicles[XVII],July 1995
1,Rivals Quick Start Set,July 1996
2,Multiverse Gift Box,November 1996
3,Anthologies,November 1998
4,Battle Royale Box Set,"November 12, 1999"


### Appending standard-legal, core, and supplemental set dfs

In [2417]:
# Appending core set frame

sets_and_rel_dates = sets_and_rel_dates.append(core_sets_and_rel_dates, ignore_index = True)
sets_and_rel_dates = sets_and_rel_dates.append(supp_sets_and_rel_dates, ignore_index = True)

sets_and_rel_dates.tail()

,expansion,rel_date
178,2014 Holiday Gift Box,"November 14, 2014"
179,2015 Holiday Gift Box,"November 6, 2015"
180,Shadows over Innistrad: The Gift Box,"May 13, 2016"
181,Kaladesh: The Gift Box,N/A
182,Throne of Eldraine Bundle Gift Edition [215],"November 15, 2019"


Adding Modern Horizons

In [2418]:
sets_and_rel_dates = sets_and_rel_dates.append({'expansion' : 'Modern Horizons' , 'rel_date' : 'June 14, 2019'} , ignore_index=True)

### Cleaning up expansion names in both frames

In [2419]:
sets_and_rel_dates['expansion'][65:90]

65                Fate Reforged
66            Dragons of Tarkir
67          Battle for Zendikar
68        Oath of the Gatewatch
69       Shadows over Innistrad
70                Eldritch Moon
71                     Kaladesh
72                Aether Revolt
73                     Amonkhet
74          Hour of Devastation
75                       Ixalan
76             Rivals of Ixalan
77                    Dominaria
78            Guilds of Ravnica
79           Ravnica Allegiance
80        War of the Spark[132]
81           Throne of Eldraine
82         Theros: Beyond Death
83    Ikoria: Lair of Behemoths
84              Zendikar Rising
85        Limited Edition Alpha
86         Limited Edition Beta
87            Unlimited Edition
88              Revised Edition
89               Fourth Edition
Name: expansion, dtype: object

In [2421]:
modern_df = modern_df.replace('Theros Beyond Death', 'Theros: Beyond Death')

In [2422]:
sets_and_rel_dates = sets_and_rel_dates.replace('War of the Spark[132]', 'War of the Spark')

In [2425]:
good = ['2010 Core Set', '2011 Core Set','2012 Core Set', '2013 Core Set', '2014 Core Set', '2015 Core Set']
bad = ['Magic 2010', 'Magic 2011', 'Magic 2012', 'Magic 2013', 'Magic 2014', 'Magic 2015',]

In [2426]:
for i, j in zip(bad, good):
    sets_and_rel_dates= sets_and_rel_dates.replace(i,j)

In [2427]:
bad2= ['Shadows over Innistrad','Modern Masters 2015 Edition', 'Eighth Edition','Ninth Edition', 'Tenth Edition', 'Ravnica: City of Guilds', 'Coldsnap[IX]']

good2=['Shadows Over Innistrad','Modern Masters 2015','8th Edition','9th Edition','10th Edition', 'Ravnica', 'Coldsnap']

In [2428]:
for i, j in zip(bad2, good2):
    sets_and_rel_dates= sets_and_rel_dates.replace(i,j)

Fixing Rise of the Eldrazi formatting -- adding year

In [2429]:
# RotE formatting

sets_and_rel_dates[45:55]

,expansion,rel_date
45,Eventide,"July 25, 2008"
46,Shards of Alara,"October 3, 2008"
47,Conflux,"February 6, 2009"
48,Alara Reborn,"April 30, 2009"
49,Zendikar,"October 2, 2009"
50,Worldwake,"February 5, 2010"
51,Rise of the Eldrazi,"April 23,"
52,Scars of Mirrodin,"October 1, 2010"
53,Mirrodin Besieged,"February 4, 2011"
54,New Phyrexia,"May 13, 2011[98]"


In [2430]:

# df.loc[df[<some_column_name>] == <condition>, [<another_column_name>]] = <value_to_add>

sets_and_rel_dates.loc[sets_and_rel_dates['expansion']=='Rise of the Eldrazi', ['rel_date']]='April 23, 2010'

### Removing N/A's

In [2432]:
sets_and_rel_dates['rel_date'].replace('N/A', np.nan)

0          December 1993
1             March 1994
2              June 1994
3            August 1994
4          November 1994
             ...        
179     November 6, 2015
180         May 13, 2016
181                  NaN
182    November 15, 2019
183        June 14, 2019
Name: rel_date, Length: 184, dtype: object

### Removing '\n'

In [2433]:
sets_and_rel_dates['rel_date'] = sets_and_rel_dates['rel_date'].apply(lambda x:x.strip())

sets_and_rel_dates['rel_date'][22]

'June 4, 2001'

### Removing [ ]

In [2434]:

sets_and_rel_dates['rel_date'][54] = sets_and_rel_dates['rel_date'][54].replace('[98]', '')

### Changing date column to datetime formate

In [2436]:
# does it work on our df?

pd.to_datetime('April 1, 2011')

pd.to_datetime('April 1994')

Timestamp('1994-04-01 00:00:00')

In [2437]:
dates_format_correct = []
for date_ in sets_and_rel_dates['rel_date']:
    try:
        dates_format_correct.append(pd.to_datetime(date_))
    except: dates_format_correct.append(np.nan)

dates_format_correct

[Timestamp('1993-12-01 00:00:00'),
 Timestamp('1994-03-01 00:00:00'),
 Timestamp('1994-06-01 00:00:00'),
 Timestamp('1994-08-01 00:00:00'),
 Timestamp('1994-11-01 00:00:00'),
 Timestamp('1995-06-01 00:00:00'),
 Timestamp('1995-10-01 00:00:00'),
 Timestamp('1996-06-10 00:00:00'),
 Timestamp('1996-10-07 00:00:00'),
 Timestamp('1997-02-03 00:00:00'),
 Timestamp('1997-06-09 00:00:00'),
 Timestamp('1997-10-13 00:00:00'),
 Timestamp('1998-03-02 00:00:00'),
 Timestamp('1998-06-15 00:00:00'),
 Timestamp('1998-10-12 00:00:00'),
 Timestamp('1999-02-15 00:00:00'),
 Timestamp('1999-06-07 00:00:00'),
 Timestamp('1999-10-04 00:00:00'),
 Timestamp('2000-02-14 00:00:00'),
 Timestamp('2000-06-05 00:00:00'),
 Timestamp('2000-10-02 00:00:00'),
 Timestamp('2001-02-05 00:00:00'),
 Timestamp('2001-06-04 00:00:00'),
 Timestamp('2001-10-01 00:00:00'),
 Timestamp('2002-02-04 00:00:00'),
 Timestamp('2002-05-27 00:00:00'),
 Timestamp('2002-10-07 00:00:00'),
 Timestamp('2003-02-03 00:00:00'),
 Timestamp('2003-05-

### Adding in corrected column

In [2438]:
sets_and_rel_dates['rel_date_correct'] = dates_format_correct

### Making dict to map on to


In [2439]:
expansion_date_dict = pd.Series(sets_and_rel_dates['rel_date_correct'].values,index=sets_and_rel_dates['expansion']).to_dict()

### Add release date column to df


In [2442]:
modern_df['rel_date'] = modern_df['expansion'].map(expansion_date_dict)
modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len,num_printings,rel_date
name,,,,,,,,,,,,,,,,
Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99,1,0,0,0,0,4,401,1,2017-04-28
Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99,0,1,0,0,0,3,234,1,2019-06-14
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99,0,0,0,0,0,3,136,1,2014-09-26
Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99,0,0,0,0,0,3,204,1,2012-05-04
Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99,0,0,0,0,0,3,140,1,2014-09-26


### Adding time_since_release column

In [2318]:
dt.date.today()

datetime.date(2020, 10, 8)

Modern

In [2443]:
modern_df['time_since_release'] = pd.to_datetime('today') - modern_df['rel_date']

In [2444]:
type(modern_df.time_since_release)

modern_df.time_since_release[0]

Timedelta('1259 days 10:38:57.953834')

Turn Timedelta type into days

In [2446]:
# modern_df['time_since_release'] = pd.to_datetime('today') - modern_df['rel_date']

modern_df['time_since_release'] = modern_df['time_since_release'].apply(lambda x: get_days(x))

In [2448]:
modern_df['time_since_release']

name
Gideon of the Trials    1259.0
Force of Negation        482.0
Polluted Delta          2204.0
Cavern of Souls         3079.0
Wooded Foothills        2204.0
                         ...  
Sky Ruin Drake          4024.0
Territorial Baloth      4024.0
Trapfinder's Trick      4024.0
Welkin Tern             4024.0
Whiplash Trap           4024.0
Name: time_since_release, Length: 16141, dtype: float64

In [2449]:
modern_df['time_since_release'].value_counts()

4836.0    363
5550.0    339
6282.0    336
818.0     293
5851.0    287
         ... 
4633.0    150
4179.0    145
4262.0    145
3534.0    145
3898.0    145
Name: time_since_release, Length: 70, dtype: int64

Some supplemental sets do not have a release date. Which ones?

In [2455]:
mask = modern_df["time_since_release"].isna()
modern_df[mask]

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len,num_printings,rel_date,time_since_release
name,,,,,,,,,,,,,,,,,
Glimpse the Unthinkable,Guilds of Ravnica: Guild Kits,R,ub,2,ub,Target player mills ten cards.,12.99,0,1,1,0,0,3,30,2,NaT,NaN
"Aurelia, the Warleader (Foil)",Guilds of Ravnica: Guild Kits,M,2rrww,6,wr,"Flying, vigilance, haste Whenever Aurelia, the...",19.99,1,0,0,1,0,4,182,1,NaT,NaN
Master of Cruelties,Ravnica Allegiance: Guild Kits,M,3br,5,rb,"First strike, deathtouch Master of Cruelties c...",10.99,0,0,1,1,0,4,226,2,NaT,NaN
Privileged Position,Guilds of Ravnica: Guild Kits,R,2www,5,w,Other permanents you control can't be the targ...,16.99,1,0,0,0,0,3,95,2,NaT,NaN
Birds of Paradise,Ravnica Allegiance: Guild Kits,R,g,1,g,: Add one mana of any color.,8.99,0,0,0,0,1,3,28,7,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Scab-Clan Mauler,Ravnica Allegiance: Guild Kits,C,rg,2,gr,Bloodthirst 2 (If an opponent was dealt damage...,0.25,0,0,0,1,1,1,135,2,NaT,NaN
Skarrgan Firebird,Ravnica Allegiance: Guild Kits,R,4rr,6,r,Bloodthirst 3 (If an opponent was dealt damage...,0.25,0,0,0,1,0,3,266,3,NaT,NaN
Sky Hussar,Ravnica Allegiance: Guild Kits,U,3wu,5,wu,"Flying When Sky Hussar enters the battlefield,...",0.25,1,1,0,0,0,2,268,2,NaT,NaN


Let's remove the nan's:

In [2456]:
modern_df.dropna(axis=0, subset=['time_since_release'], inplace=True)

modern_df.head()

,expansion,rarity,cost,cmc,color_identity,rules_text,price,is_w,is_u,is_b,is_r,is_g,rarity_num,rules_text_len,num_printings,rel_date,time_since_release
name,,,,,,,,,,,,,,,,,
Gideon of the Trials,Amonkhet,M,1ww,3,w,"[+1]: Until your next turn, prevent all damage...",6.99,1,0,0,0,0,4,401,1,2017-04-28,1259.0
Force of Negation,Modern Horizons,R,1uu,3,u,"If it's not your turn, you may exile a blue ca...",54.99,0,1,0,0,0,3,234,1,2019-06-14,482.0
Polluted Delta,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Polluted Delta: Search...",32.99,0,0,0,0,0,3,136,1,2014-09-26,2204.0
Cavern of Souls,Avacyn Restored,R,,0,colorless,"As Cavern of Souls enters the battlefield, cho...",79.99,0,0,0,0,0,3,204,1,2012-05-04,3079.0
Wooded Foothills,Khans of Tarkir,R,,0,colorless,", Pay 1 life, Sacrifice Wooded Foothills: Sear...",29.99,0,0,0,0,0,3,140,1,2014-09-26,2204.0


# Exporting modern_df as pickle for use in accompanying notebook.

See accompanying notebook: 'Project_2_modeling_final'.

In [2457]:
filename = 'modern_df_ready_for_model_10-8-20_PICKLE'
outfile = open(filename,'wb')
pickle.dump(modern_df,outfile)
outfile.close()
